In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## Question 1

In [ ]:
print(spark.version)

## Question 2

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [ ]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [ ]:
df = df.repartition(4)

In [ ]:
df.write.parquet('partitions/')

In [ ]:
!du -ah "partitions" | grep '\.parquet$'

## Question 3

In [ ]:
df = spark.read.parquet('partitions/')

In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView('trips')
spark.sql("""
          SELECT
            count(*)
          FROM trips
          WHERE
            date(tpep_pickup_datetime) = '2024-10-15'""").show()

## Question 4

In [ ]:
spark.sql("""
          SELECT
            *,
            (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS duration
          FROM trips
          ORDER BY duration DESC
          LIMIT 1
          """).show()

## Question 6

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [ ]:
df_lookup = spark.read.csv('taxi_zone_lookup.csv', header=True)
df_lookup.createOrReplaceTempView('lookup')
df_lookup.printSchema()

In [ ]:
spark.sql("""
          SELECT
            Zone,
            count(PULocationID)
          FROM trips
          JOIN lookup
          on trips.PULocationID = lookup.LocationID
          GROUP BY Zone
          ORDER BY count(PULocationID) ASC
          LIMIT 5
          """).show(truncate=False)